# Input data requirements

The input ice sheet model should be a netCDF file. 


### `Lithk` variable
The uploaded model to contain thickness data (the `lithk` variable) for the comparison.


In [1]:
import os,sys


# Get the current working directory
current_directory = os.getcwd()
# Navigate two levels up to reach main dir
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir, os.pardir))
# Insert the specific directory into the system path
sys.path.insert(0, os.path.join(parent_directory, 'bin', 'IMBIE'))
from imbie_utils import *

# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')



### Configure IMBIE comparison

In [2]:
# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "AIS" # Change to "AIS" or "GIS"

# Define the flag for the area region Basin mask or Total area
area ="Basin" # Change to "Basin" or "Total"

# Set start and end dates
start_date = '2007-01-01'
end_date ='2009-01-01'

#Set density of ice
rho_ice = 918 # (kg/m^3)

#output file dirctory
output_path='/home/jovyan/CmCt/notebooks/IMBIE/'

# Select  variable for mass balance comparision
mass_balance_column="Cumulative dynamics mass balance anomaly (Gt)"# "Cumulative dynamics mass balance anomaly (Gt)"
if mass_balance_column == "Cumulative mass balance (Gt)":
    mass_balance_type = "total"
elif mass_balance_column == "Cumulative dynamics mass balance anomaly (Gt)":
    mass_balance_type = "dynamic"


# Set model model path,shapefile path and projection and IMBIE csv_file
if icesheet == "GIS":
    projection = "EPSG:3413"  # Greenland
    #Set the model data dir path
    # nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_JPL_ISSM_asmb.nc'
    nc_filename='/home/jovyan/shared-public/CmCt/models/ISMIP6/lithk_GIS_GSFC_ISSM_historical.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    #Set the observation data dir path
    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2022_Gt_partitioned_v0.csv'
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_greenland_2021_Gt.csv'

    ##Set the Region observation data dir path
    obs_east_filename = None
    obs_west_filename = None
    obs_peninsula_filename = None
    
    
elif icesheet== "AIS":
    projection = "EPSG:3031"  # Antarctica    
    #Set the model data dir path
    nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_AIS_AWI_PISM1_hist_std.nc'
    #Set the shape data dir path
    shape_filename = "/home/jovyan/CmCt/data/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    #Set the observation data dir path
    obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2022_Gt_partitioned_v0.csv'
    
    
    ##Set the Region observation data dir path
    obs_east_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_east_antarctica_2022_Gt_partitioned_v0.csv'
    obs_west_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_west_antarctica_2022_Gt_partitioned_v0.csv'
    obs_peninsula_filename= '/home/jovyan/CmCt/data/IMBIE/imbie_antarctic_peninsula_2022_Gt_partitioned_v0.csv'


    # obs_filename = '/home/jovyan/CmCt/data/IMBIE/imbie_antarctica_2021_Gt.csv'
    # ##Set the Region observation data dir path
    # obs_east_filename = None
    # obs_west_filename = None
    # obs_peninsula_filename = None


else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")

In [3]:
# Check if  observation file exists
if not os.path.exists(obs_filename):
    raise FileNotFoundError(f"Observation file not found: {obs_filename}")

# Check if model file exists    
if not os.path.exists(nc_filename):
    raise FileNotFoundError(f"Model file not found: {nc_filename}")


if icesheet== "AIS":
    if (obs_east_filename and os.path.exists(obs_east_filename)) and \
           (obs_west_filename and os.path.exists(obs_west_filename)) and \
           (obs_peninsula_filename and os.path.exists(obs_peninsula_filename)):
        # Check if regional observation files exist 
        if not os.path.exists(obs_east_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_east_filename}")
        if not os.path.exists(obs_west_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_west_filename}")
        if not os.path.exists(obs_peninsula_filename):
            raise FileNotFoundError(f"Observation file not found: {obs_peninsula_filename}")

## Mass change comparision processing

In [4]:
gis_ds = xr.open_dataset(nc_filename)
time_var = gis_ds['time']

calendar_type = time_var.to_index().calendar
start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d')
end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Adjust day to be 30 ( to avoid error if it's the 31st day in a 360_day calendar)
start_date_cftime = cftime.datetime(start_date_dt.year, start_date_dt.month, min(start_date_dt.day, 30), calendar=calendar_type)    

end_date_cftime = cftime.datetime(end_date_dt.year, end_date_dt.month, min(end_date_dt.day, 30), calendar=calendar_type)  

start_date_fract = start_date_cftime.year + (start_date_cftime.dayofyr-1) / days_in_year(start_date_cftime)
end_date_fract = end_date_cftime.year + (end_date_cftime.dayofyr-1) / days_in_year(end_date_cftime)


In [5]:
# Calculate time-varying mass balance change of IMBIE from start to end date
# #IMBIE total mass balance
IMBIE_total_mass_change_sum=sum_MassBalance(obs_filename,start_date_fract,end_date_fract,mass_balance_column)


# Process the model data,calculate time-varying  model mass balance change from IMBIE's start to end date 
basin_result=process_model_data(nc_filename,IMBIE_total_mass_change_sum,start_date_cftime, end_date_cftime,start_date_fract, end_date_fract,rho_ice,projection,shape_filename,icesheet)


# Calculate difference of IMBIE-model mass change
results=process_IMBIE(obs_filename, start_date_fract, end_date_fract, icesheet, basin_result,IMBIE_total_mass_change_sum,mass_balance_column,obs_east_filename, obs_west_filename, obs_peninsula_filename)

The selected dates 2007-01-01 00:00:00 and 2009-01-01 00:00:00 are within the range of the model data.


## Display result and write it to csv file

In [6]:
# Extract the base name of the nc file (without .nc extension)
nc_base_filename = os.path.basename(nc_filename).replace('.nc', '')

# Create the CSV filename by combining the output path and the base nc filename with .csv extension
csv_filename = os.path.join(output_path, f"{nc_base_filename}.csv")

write_and_display_mass_change_comparison_all_dates(icesheet, basin_result, results, mass_balance_type, start_date_fract,end_date_fract, csv_filename)    # Initialize list to store rows of data for CSV
   


 Time-varying Mass change comparison (dynamic): 2007.0 - 2009.0
Date            Basin/Region         Model mass change (Gt)    IMBIE mass change (Gt)    Residual (Gt)       
2007.0          Masked_Total         0.0                       0.0                       0.0                 
2007.0833       Masked_Total         -6.69                     -7.39                     -0.70               
2007.1667       Masked_Total         -13.38                    -14.77                    -1.39               
2007.25         Masked_Total         -20.07                    -22.16                    -2.09               
2007.3333       Masked_Total         -26.75                    -29.55                    -2.80               
2007.4167       Masked_Total         -33.45                    -36.93                    -3.49               
2007.5          Masked_Total         -40.13                    -44.32                    -4.19               
2007.5833       Masked_Total         -46.82            